## Step 2: Add Vehicle Assignments

Follows Step 1, `add_warnings_with_status_time_open_to_db.ipynb`
Followed by Step 3, 
**Successfully run on complete historical data, adding vehicle assignment table to `ituran_synchromatics_data.sqlite` in Data Integration - All Months**

create a driver_schedule table, and
for each month, and for each route, add all records to that single table

we may care to sort the records before adding to the database table

first, we need to know if it is safe to use vehicle_assignment_id as the
primary key for driver schedule records, so we test for uniqueness across all
data files: for each VehiclesThatRanRoute file across all routes and months,
read vehicle_assignment_id values into an array, count the unique array
entries and compare for equality with the array length.

In [1]:
import numpy as np
from os import path, walk
import pandas as pd
from sqlalchemy import create_engine

# Search over .zip files in LADOT Synchromatics data
project_root_dir = r'\\vntscex.local\DFS\3BC-Share$_Mobileye_Data\Data\Data Integration - All Months' 
data_root_dir = path.join(project_root_dir, 'data_sources')

vehicle_assignment_data = []

In [2]:
for dir, subdirs, files in walk(data_root_dir):
    print(files)

[]
[]
['296_296DASHB_RouteAndStops.txt', '296_296DASHB_StopTimes_100118_to_103118.txt', '296_296DASHB_VehiclesThatRanRoute_100118_to_103118.txt']
['297_297DASHE_RouteAndStops.txt', '297_297DASHE_StopTimes_100118_to_103118.txt', '297_297DASHE_VehiclesThatRanRoute_100118_to_103118.txt']
['298_298DASHF_RouteAndStops.txt', '298_298DASHF_StopTimes_100118_to_103118.txt', '298_298DASHF_VehiclesThatRanRoute_100118_to_103118.txt']
['9212_9212DASHD_RouteAndStops.txt', '9212_9212DASHD_StopTimes_100118_to_103118.txt', '9212_9212DASHD_VehiclesThatRanRoute_100118_to_103118.txt']
['9960_9960DASHA_StopTimes_100118_to_103118.txt', '9960_9960DASHA_VehiclesThatRanRoute_100118_to_103118.txt']
[]
['296_296DASHB_RouteAndStops.txt', '296_296DASHB_StopTimes_090118_to_093018.txt', '296_296DASHB_VehiclesThatRanRoute_090118_to_093018.txt']
['297_297DASHE_RouteAndStops.txt', '297_297DASHE_StopTimes_090118_to_093018.txt', '297_297DASHE_VehiclesThatRanRoute_090118_to_093018.txt']
['298_298DASHF_RouteAndStops.txt', 

In [3]:
for dir, subdirs, files in walk(data_root_dir):
  # we assume that files only exist at the nodes
  if len(files) > 0:
    # we assume that only one driver schedule file exists in the current dir
    try:
      file_name_indices = [
        file.find('_VehiclesThatRanRoute_') >= 0 for file in files]

      file_name_index = file_name_indices.index(True)
      file_name = files[file_name_index]
      file_path = path.join(dir, file_name)

      # forget using np.unicode_ for strings since pandas treats them as objects
      # we can specify the data type since none of the values are null
      df = pd.read_table(
        file_path, usecols=[0, 1, 2, 3, 5, 6, 11, 12, 13, 14],
        header=None, skiprows=[0], parse_dates=['start_time', 'end_time'],
        names=['vehicle_assignment_id', 'vehicle_id', 'route_id', 'driver_id',
               'start_time', 'end_time', 'bus_number', 'first_name',
               'last_name', 'badge_number'],
        dtype={'vehicle_assignment_id': np.uint64, 'vehicle_id': np.uint32,
               'route_id': np.uint32, 'driver_id': np.uint32,
               'start_time': object, 'end_time': object,
               'bus_number': np.uint32, 'first_name': object,
               'last_name': object, 'badge_number': np.uint32})

      print(df.head(2))
      print(df.dtypes)

      vehicle_assignment_data.append(df)
    except Exception as e:
      print('Driver schedule file not found in {}'.format(dir))
      print(e)
      continue

   vehicle_assignment_id  vehicle_id  route_id  driver_id          start_time  \
0                1049661        1615       296       1977 2018-10-01 05:23:51   
1                1049624        1609       296       2010 2018-10-01 05:28:36   

             end_time  bus_number first_name last_name  badge_number  
0 2018-10-01 09:57:16       15328    Richard     Trejo          6004  
1 2018-10-01 08:42:04       15322      Juana  Calderon          6042  
vehicle_assignment_id            uint64
vehicle_id                       uint32
route_id                         uint32
driver_id                        uint32
start_time               datetime64[ns]
end_time                 datetime64[ns]
bus_number                       uint32
first_name                       object
last_name                        object
badge_number                     uint32
dtype: object
   vehicle_assignment_id  vehicle_id  route_id  driver_id          start_time  \
0                1051929         347       297  

In [4]:
vehicle_assignment_data = pd.concat(
  vehicle_assignment_data, ignore_index=True, verify_integrity=True)

# records of runs that span two days may appear once for each day depending on
# how the Excel exports were preformed, and should be dropped
vehicle_assignment_data.drop_duplicates(inplace=True)

# we temporarily also drop records with missing values to prove our concept.
# Key attributes that require values include 1) vehicle_assessment_id,
# 2) vehicle_id, 3) BusNumber, 4) driver_id (at least for longitudinal),
# 5) start_time, and 6) end_time.
# TODO: Infer missing values where possible using warning and route data
key_column_names = ['vehicle_assignment_id', 'vehicle_id', 'bus_number',
                    'driver_id', 'start_time', 'end_time']

vehicle_assignment_data.dropna(subset=key_column_names, inplace=True)

# we make no assumption about the order in which source xlsx files are input
vehicle_assignment_data.sort_values(['start_time', 'end_time'], inplace=True)

# after removing duplicate records, vehicle_assignment_ids will be unique and
# can be used as the primary key of the vehicle_assignment table. Because we
# don't yet know how this results in a SQLite PK, just reset the indices for now
# vehicle_assignment_data.set_index('vehicle_assignment_id', inplace=True)
vehicle_assignment_data.set_index(
  pd.RangeIndex(vehicle_assignment_data.shape[0]), inplace=True)

print(vehicle_assignment_data.describe())
print(df.dtypes)

       vehicle_assignment_id    vehicle_id      route_id     driver_id  \
count           2.781900e+04  27819.000000  27819.000000  27819.000000   
mean            1.040284e+06   1488.457673   4024.907114   2967.038032   
std             2.259969e+04   1009.710812   4389.401522   2980.353233   
min             9.848530e+05    303.000000    296.000000     64.000000   
25%             1.025970e+06    681.500000    297.000000   1982.000000   
50%             1.041994e+06   1610.000000    298.000000   2016.000000   
75%             1.059384e+06   1631.000000   9212.000000   2111.000000   
max             1.074193e+06   4386.000000   9960.000000  13336.000000   

         bus_number   badge_number  
count  27819.000000   27819.000000  
mean   14661.530716   76146.091089  
std     1797.373955  144419.267690  
min     6303.000000     801.000000  
25%    12334.000000    6040.000000  
50%    15323.000000    6128.000000  
75%    15343.000000    7410.000000  
max    17312.000000  379959.000000  


In [5]:
# db_path = 'sqlite:///ituran_synchromatics_data.sqlite'
db_path = 'sqlite:///' + path.join(project_root_dir, 'ituran_synchromatics_data.sqlite')

db = create_engine(db_path)

# poor performance has been observed when adding more than one million records
# at a time
vehicle_assignment_data.to_sql(
  'vehicle_assignment', db, if_exists='replace', chunksize=1000000, index=False)